Source: https://github.com/samwit/langchain-tutorials/blob/main/RAG/YT_Chat_your_PDFs_Langchain_Template_for_creating.ipynb

In [1]:
# Loads .env variables
%load_ext dotenv
%dotenv

## Basic Chat PDF

We'll use `CharacterTextSplitter` to split the document into chunks, then convert these chunks into embeddings with `OpenAIEmbeddings`, and finally store these embedding vectors with `FAISS` to perform similarity searches

**[OpenAIEmbeddings](https://api.python.langchain.com/en/latest/embeddings/langchain.embeddings.openai.OpenAIEmbeddings.html)**
* OpenAI embedding model, necessary when using other OpenAI models
* [OpenAI Embeddings docs](https://platform.openai.com/docs/guides/embeddings/what-are-embeddings)


**[FAISS (Facebook AI Similarity Search)](https://python.langchain.com/docs/integrations/vectorstores/faiss):**
* Library for efficient similarity search and clustering of dense vectors.
* It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM.
* It also contains supporting code for evaluation and parameter tuning.
* [FAISS article](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/)

In [2]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS 
from tqdm import tqdm

## Reading the PDF

In [3]:
file_name = "Rodolitos_recifes_peixes.pdf"
# file_name = "Bianchi, 2021_Estimating global biomass fishing_Science.pdf"

doc_reader = PdfReader(f"./articles/{file_name}")
doc_reader

In [4]:
# read data from the file and put them into a variable called raw_text
raw_text = ''
for i, page in enumerate(tqdm(doc_reader.pages)):
    text = page.extract_text()
    if text:
        raw_text += text

print(f"{i+1} pages merged into {len(raw_text)} characters")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 25.22it/s]

10 pages merged into 43865 characters


In [5]:
print(raw_text[:1000])

1
Vol.:(0123456789) Scientific Reports  |          (2021) 11:794  | https://doi.org/10.1038/s41598-020-80574-w
www.nature.com/scientificreportsTropical rhodolith beds are a major 
and belittled reef fish habitat
Rodrigo L. Moura1,7*, Maria L. Abieri1,7, Guilherme M. Castro1, Lélis A. Carlos‑Júnior1, 
Pamela M. Chiroque‑Solano1, Nicole C. Fernandes1, Carolina D. Teixeira1, Felipe V. Ribeiro1, 
Paulo S. Salomon1, Matheus O. Freitas1, Juliana T. Gonçalves1, Leonardo M. Neves2, 
Carlos W. Hackradt3, Fabiana Felix‑Hackradt3, Fernanda A. Rolim4, Fábio S. Motta5, 
Otto B. F. Gadig4, Guilherme H. Pereira‑Filho5 & Alex C. Bastos6
Understanding habitat ‑level variation in community structure provides an informed basis for natural 
resources’ management. Reef fishes are a major component of tropical marine biodiversity, but their 
abundance and distribution are poorly assessed beyond conventional SCUBA diving depths. Based on 
a baited‑video survey of fish assemblages in Southwestern Atlantic’s m

## Text Splitter

Takes the full text and splits it into chunks, for indexing. The chunk size is measured in characters, not tokens

In [6]:
text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200, # text overlap between chunks
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

len(texts)

54

In [8]:
print(texts[10])

less exposed to fisheries and land-based stressors. Our standardized survey with BRUVs allowed for richness and 
biomass estimates for nearshore and mid-shelf reefs, as well as rhodolith beds in depths beyond SCUBA limits. 
Instead of being marginal (i.e. “suboptimal”)  habitats25,26, rhodolith beds were found to be major reef fish habitats 
in the tropical SW A and need to be thoughtfully accounted for conservation planning and marine management.
Results
We recorded 107 reef fish species (5,155 individuals), 71 (66.4%) in fringing and pinnacles’ reefs and 85 (79.4%) 
in rhodolith beds (Supplementary Table S1 online). The same richness rank between the two megahabitats was 
obtained with rarefaction and extrapolation-based estimates (Supplementary Fig. S1 online). Nearly half [49] of 
all species were habitat generalists that occurred in both megahabitats. Unique occurrences were concentrated in


## Making the embeddings

In [9]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [10]:
# Creates a FAISS vector store using the article chunks and the OpenAI embeddding
docsearch = FAISS.from_texts(texts, embeddings)

In [25]:
query = "Discussion"
docs = docsearch.similarity_search(query)

print(len(docs))
print(docs)

4
[Document(page_content='Paulista, São Vicente, SP , Brazil. 5Laboratório de Ecologia e Conservação Marinha, Instituto Do Mar, Universidade \nFederal de São Paulo, Santos, SP , Brazil. 6Universidade Federal do Espírito Santo, Vitória, ES, Brazil. 7These authors \ncontributed equally: Rodrigo L. Moura and Maria L. Abieri. *email: moura.uesc@gmail.com2\nVol:.(1234567890) Scientific Reports  |          (2021) 11:794  | https://doi.org/10.1038/s41598-020-80574-w\nwww.nature.com/scientificreports/among several other fish families, engage into long-range movements towards nearshore or offshore spawning \n grounds19, and mangroves and seagrass beds can be important habitats for early stages of reef  fishes3,4. On short \ntime scales, diel movements of diurnal planktivores (e.g. damselfishes) and nocturnal invertivores (e.g. grunts \nand emperor breams) enhance the coupling among benthic habitats and the water column [e.g.,3]. Knowledge', metadata={}), Document(page_content='Most of the varia

## Plain QA Chain

In [17]:
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

In [47]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",  # default model is being deprecated and is more expensive
    temperature=0
)

chain = load_qa_chain(
    llm, 
    chain_type="stuff"
)

In [48]:
# check the default LLM chain prompt
chain.llm_chain.prompt

ChatPromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], output_parser=None, partial_variables={}, template="Use the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}", template_format='f-string', validate_template=True), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], output_parser=None, partial_variables={}, template='{question}', template_format='f-string', validate_template=True), additional_kwargs={})])

In [49]:
query = "What are tropical rodolith beds?"

# Ask ChatGPT the question sending the most similar pages as input_documents
docs = docsearch.similarity_search(query)
answer = chain.run(input_documents=docs, question=query)
print(f"{query}\n")
print(f"{answer}\n")

# Ask ChatGPT the question without sending the input_documents
answer = chain.run(input_documents=[], question=query)
print(f"{query}\n")
print(f"{answer}\n")

What are tropical rodolith beds?

Tropical rhodolith beds are extensive benthic systems dominated by algae. They are found along the tropical continental shelf and are one of the largest algae-dominated benthic systems in the world. Rhodolith beds are characterized by the presence of rhodoliths, which are nodules formed by the accumulation of calcareous algae. These beds can be found at depths ranging from 10 to 150 meters. Rhodolith beds are considered a major but often overlooked habitat for reef fish and play a significant role in the biodiversity and ecological processes of tropical marine ecosystems.

What are tropical rodolith beds?

Tropical rodolith beds are underwater ecosystems found in tropical regions. They are composed of small, free-living, calcareous red algae known as rodoliths. These rodoliths form dense beds on the seafloor, creating a unique habitat for a variety of marine organisms. The rodoliths provide shelter, food, and a substrate for attachment for many species

In [37]:
query = "\nSummarize key aspects about rodolith beds"
docs = docsearch.similarity_search(query)

# Ask ChatGPT the question sending the most similar pages as input_documents
answer = chain.run(input_documents=docs, question=query)
print(f"{query}\n")
print(f"{answer}\n")

# Ask ChatGPT the question without sending the input_documents
answer = chain.run(input_documents=[], question=query)
print(f"{query}\n")
print(f"{answer}\n")


Summarize key aspects about rodolith beds

Rhodolith beds are a type of benthic habitat found along the tropical continental shelf. They are one of the world's largest algae-dominated benthic systems. Rhodolith beds have a larger area and broader depth and cross-shelf range compared to reefs. They are characterized by high richness of reef fishes, although fish biomass is smaller in rhodolith beds compared to reefs. Rhodolith beds play a significant role in the conservation of biodiversity, particularly for endemic species. However, they are often overlooked and undervalued in terms of their importance for conservation. Rhodolith beds are currently facing threats from activities such as carbonates mining and oil and gas exploitation. The trophic structure of fish assemblages in rhodolith beds is distinct, with lower abundance of herbivores compared to reefs. Rhodolith beds may influence ecological processes and drive reef fish community structure and dynamics. Overall, rhodolith beds 

In [56]:
# Test sending the whole article to the model
from langchain.docstore.document import Document
doc =  Document(page_content=raw_text, metadata={"source": "local"})

query = "\nWhat part of the article could be used in a oceanography master's degree test?"
answer = chain.run(input_documents=[doc], question=query)
print(f"{query}\n")
print(f"{answer}\n")

InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 11478 tokens. Please reduce the length of the messages.